In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")

In [21]:
from springstone.data import get_data, get_missing_dates, create_train_test, create_df_for_prophet, download_model
from springstone.utils import bollinger_bands, moving_average, prophet_preprocessing, prophet_non_business_days
from springstone.utils import basic_recommendation, temp_data_predict
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.plot import plot_forecast_component_plotly
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import joblib
from termcolor import colored
from springstone.utils import temp_data_predict
from datetime import date, timedelta
import holidays

In [14]:
ticker = 'AAPL'

In [3]:
hist = get_data(ticker)

[*********************100%***********************]  1 of 1 completed


In [4]:
hist

,Open,High,Low,Close,Volume
Date,,,,,
2012-02-21,18.102858,18.387501,18.004286,18.387501,605595200
2012-02-22,18.324286,18.410357,18.181070,18.322857,483302400
2012-02-23,18.395714,18.493929,18.196428,18.442499,568027600
2012-02-24,18.559643,18.674999,18.522858,18.657499,415072000
2012-02-27,18.618214,18.875000,18.438572,18.777143,547582000
...,...,...,...,...,...
2022-02-14,167.369995,169.580002,166.559998,168.880005,86185500
2022-02-15,170.970001,172.949997,170.250000,172.789993,64286300
2022-02-16,171.850006,173.339996,170.050003,172.550003,61177400


In [5]:
df_prophet = prophet_preprocessing(hist, 'Close')

In [6]:
non_business_days_df = prophet_non_business_days(hist)

In [7]:
df_train, df_test = create_train_test(df_prophet)

In [ ]:
df_train

In [ ]:
df_test

In [8]:
m_prophet = Prophet(holidays=non_business_days_df)
m_prophet.fit(df_train)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -80.3622


    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       7267.21     0.0517129       1160.68       1.166      0.1166      121   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       7303.37     0.0106924       456.687       2.431     0.02431      237   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       7311.95     0.0102775       774.401           1           1      352   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       7319.48    0.00887635       625.283      0.5338           1      477   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       7322.74   0.000563668       90.4905           1           1      602   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     538     

In [9]:
joblib.dump(m_prophet, 'model_prophet_AAPL.joblib')

model.joblib saved locally


In [20]:
basic_recommendation(ticker)

[*********************100%***********************]  1 of 1 completed


'Buy recommendation'

In [ ]:
n_days_prediction=1
future = pipeline.make_future_dataframe(periods=n_days_prediction)
future

In [ ]:
forecast = pipeline.predict(future)

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'non business day']]

In [ ]:
m_prophet.plot(forecast);

In [ ]:
m_prophet.plot_components(forecast);

In [ ]:
#plot_forecast_component_plotly(m_prophet, forecast, 'non business day')

In [ ]:
forecast = forecast[- n_days_prediction:][['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'non business day']]

In [ ]:
forecast

In [ ]:
df_test[:n_days_prediction]

In [ ]:
mae_prophet = mean_absolute_error(df_test[:n_days_prediction]['y'], forecast['yhat'])
mape_prophet = mean_absolute_percentage_error(df_test[:n_days_prediction]['y'], forecast['yhat'])
print(f'MAE: {mae_prophet}')
pbrint(f'MAPE: {mape_prophet}')

In [ ]:
future = pd.DataFrame({'ds': [date(2020, 2, 20)]})
forecast = pipeline.predict(future)
y_predict = forecast[-1:]#['yhat']
y_predict

In [ ]:
hist = get_data(ticker)
hist_predict = hist[-40:]
hist_predict.loc['predict'] = [0, 0, 0, y_predict, 0]
hist_predict = bollinger_bands(hist_predict, 'Close', 20, 2)
hist_predict = bollinger_bands(hist_predict, 'Close', 20, -2)
hist_predict = moving_average(hist_predict, 'Close', 7)
list_temp = hist_predict[-1:].values.to_dict('list')

In [22]:
us_holidays = holidays.US()

In [26]:
us_holidays['2022-01-01': '2022-12-31']

[datetime.date(2022, 1, 1),
 datetime.date(2022, 1, 17),
 datetime.date(2022, 2, 21),
 datetime.date(2022, 5, 30),
 datetime.date(2022, 6, 19),
 datetime.date(2022, 6, 20),
 datetime.date(2022, 7, 4),
 datetime.date(2022, 9, 5),
 datetime.date(2022, 10, 10),
 datetime.date(2022, 11, 11),
 datetime.date(2022, 11, 24),
 datetime.date(2022, 12, 25),
 datetime.date(2022, 12, 26)]